# Visual Layer SDK Explorer

This notebook shows you how to use Visual Layer SDK to upload, explore, and filter datasets. 

**Prerequisite**

You need to have valid Visual Layer account in order to be able to interact with Visual Layer service. 
Once you have an account, you will need to get VISUAL_LAYER_API_KEY and VISUAL_LAYER_API_SECRET and put it in environment variables.

In [ ]:
# uncomment this to install necessary packages
# !pip install matplotlib
# !pip install visual-layer-sdk -i https://test.pypi.org/simple/

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display
from visual_layer_sdk import VisualLayerClient

VL_API_KEY = os.getenv('VISUAL_LAYER_API_KEY')
VL_API_SECRET = os.getenv('VISUAL_LAYER_API_SECRET')

if VL_API_KEY is None or VL_API_SECRET is None:
    raise EnvironmentError("Please configure VISUAL_LAYER_API_KEY and VISUAL_LAYER_API_SECRET environment variable.")

# Initialize the client and do a quick health check
client = VisualLayerClient(api_key=VL_API_KEY, api_secret=VL_API_SECRET)
print(f"client status: {client.healthcheck()}")

In [ ]:
# utility function to display images in a given dataset with input image IDs
def display_images(ds, image_uris):
    for index, image_uri in enumerate(image_uris):
        image_url = ds.get_image_info(image_uri)['image_uri']
        display(Image(url=image_url))

# Basic Dataset Exploration APIs

In [ ]:
# Get all datasets
datasets = client.get_all_datasets()
print(f"Found {len(datasets)} datasets")

In [ ]:
# Get a specific dataset by dataset ID
ds_id = datasets.iloc[0]['id']
ds = client.get_dataset_object(ds_id)
details = ds.get_details()
print(f"Dataset details: {details}")

In [ ]:
# Get a specific dataset by dataset ID, the 'Beans Enriched' dataset is a public dataset you can explore
LEAF_DATASET_ID = 'bc41491e-78ae-11ef-ba4b-8a774758b536' 
leaf_ds = client.get_dataset_object(LEAF_DATASET_ID)

# export all images in the dataset to a dataframe
leaf_df = leaf_ds.export_to_dataframe()

# display first 2 images
display_images(leaf_ds, leaf_df.iloc[:2]['media_id'])

# Explore different filtering functionalities

In [ ]:
# Search datasets by labels
# Find the leaf with spot and bean rust
filtered_df = leaf_ds.search_by_labels_to_dataframe(labels=['angular_leaf_spot', 'bean_rust'])

# display first 2 images
display_images(leaf_ds, filtered_df.iloc[:2]['media_id'])

In [ ]:
# Search datasets by captions
# find the data with a person holding leaf
# try other text: "blue nail"
filtered_df = leaf_ds.search_by_captions_to_dataframe(caption_text=['a person holding leaf'])

display_images(leaf_ds, filtered_df.iloc[:2]['media_id'])

# Let's explore another dataset and differnt filtering/search functions

In [ ]:
# Search datasets by issue types
PET_DATASET_ID = '3972b3fc-1809-11ef-bb76-064432e0d220'
pet_dataset = client.get_dataset_object(PET_DATASET_ID)

# fild dataset that are blur
filtered_df = pet_dataset.search_by_issues_to_dataframe(issue_type='blur')
display_images(pet_dataset, filtered_df.iloc[:2]['media_id'])

# try do a similarity search by uploading a file from your laptop

In [ ]:
# Similarity search, upload a file from your local folder 
local_image = os.path.join(os.getcwd(), 'dog.jpeg')
                      
similar_images_df = pet_dataset.search_by_visual_similarity_to_dataframe(image_path=local_image)
display_images(pet_dataset, similar_images_df.iloc[:2]['media_id'])

In [ ]:
# For advanced search, you can use the vql language. Please refer to the documentation for detailed syntax

# Search cats in the dataset
cats_df = pet_dataset.search_by_vql([{'id': 'label_filter', 'labels': {'op': 'one_of', 'value': ['cat']}}])
display_images(pet_dataset, cats_df.iloc[:2]['media_id'])